## Домашнее задание

Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов

In [1]:
import numpy as np
import pandas as pd

In [2]:
TEXT_PATH = 'evgenyi_onegin.txt'

In [3]:
text = open(TEXT_PATH, 'rb').read().decode(encoding='utf-8')

In [4]:
from nltk.tokenize import word_tokenize

In [5]:
corpus = word_tokenize(text)

In [6]:
max_words = 10000

In [7]:
from nltk.probability import FreqDist
dist = FreqDist(corpus)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [8]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [9]:
idx2word = {k: v for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [10]:
data = [vocabulary[token] for token in corpus if token in vocabulary]

In [11]:
import tensorflow as tf

In [12]:
seq_length = 100
examples_per_epoch = len(data)//(seq_length+1)

dataset = tf.data.Dataset.from_tensor_slices(data)

for i in dataset.take(5):
    print(idx2word[i.numpy()])

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

Александр
Сергеевич
Пушкин
Евгений
Онегин


2022-09-03 19:57:27.640559: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-03 19:57:27.641022: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [13]:
sequences = dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(' '.join([idx2word[idx] for idx in item.numpy()])))

'Александр Сергеевич Пушкин Евгений Онегин Роман в стихах Не мысля гордый свет забавить , Вниманье дружбы возлюбя , Хотел бы я тебе представить Залог достойнее тебя , Достойнее души прекрасной , Святой исполненной мечты , Поэзии живой и ясной , Высоких дум и простоты ; Но так и быть - рукой пристрастной Прими собранье пестрых глав , Полусмешных , полупечальных , Простонародных , идеальных , Небрежный плод моих забав , Бессонниц , легких вдохновений , Незрелых и увядших лет , Ума холодных наблюдений И сердца горестных замет . ГЛАВА ПЕРВАЯ И жить торопится и чувствовать спешит . Кн . Вяземский .'
"I `` Мой дядя самых честных правил , Когда не в шутку занемог , Он уважать себя заставил И лучше выдумать не мог . Его пример другим наука ; Но , боже мой , какая скука С больным сидеть и день и ночь , Не отходя ни шагу прочь ! Какое низкое коварство Полуживого забавлять , Ему подушки поправлять , Печально подносить лекарство , Вздыхать и думать про себя : Когда же черт возьмет тебя ! '' II Так

In [14]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [15]:
BATCH_SIZE = 64
BUFFER_SIZE = 15000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int32, name=None), TensorSpec(shape=(64, 100), dtype=tf.int32, name=None))>

In [16]:
VOCAB_SIZE = len(vocabulary)
EMBEDDING_DIM = 256
RNN_UNITS = 1024

In [17]:
class RNNgenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, batch_size):
        super(RNNgenerator, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
                                 
        self.gru1 = tf.keras.layers.GRU(RNN_UNITS,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(RNN_UNITS,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
                           
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        emb_x = self.emb(x)
        x1 = self.gru1(emb_x)
        x = x1
        for _ in range(3):
            x = self.gru2(x)
        #x = self.gru1(x)
        x = (x + x1)/2
        return self.fc(x)

model = RNNgenerator(VOCAB_SIZE, EMBEDDING_DIM, BATCH_SIZE)

In [18]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [19]:
model.compile(optimizer='adam', loss=loss)

In [20]:
EPOCHS = 50

In [44]:
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/50


2022-09-03 19:56:23.915820: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-03 19:56:24.251860: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-03 19:56:24.711883: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-03 19:56:25.999758: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-03 19:56:29.560010: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-03 19:56:31.065791: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-03 19:56:36.658172: I tensorflow/core/grappler/optimizers/cust

__!!!__

Я в истерике, у меня модель не хочет обучаться, вместо этого строчит `Plugin optimizer for device_type GPU is enabled`

In [21]:
TEMPERATURE = 0.1

def generate_text(model, start_str):
    num_iter = 80
    inputs = tf.expand_dims([vocabulary[tok] for tok in word_tokenize(start_str)], 0)
    generated_text = []
    model.reset_states()
    for i in range(num_iter):
        pred = model(inputs)
        pred = tf.squeeze(pred, 0)
        pred /= TEMPERATURE
        predicted_id = tf.random.categorical(pred, num_samples=1)[-1, 0].numpy()
        inputs = tf.expand_dims([predicted_id], 0)
        generated_text.append(idx2word[predicted_id])
    return start_str + ' '.join(generated_text)

In [22]:
gen_text = generate_text(model, start_str=u'И вот идет уже')
print(gen_text)

И вот идет уженаших Прослыть небесной необузданных грудь Байрона но прикрасой везде тихий чувству Полужуравль мой А обуза склад вынулось жужжит замирать элегии зале Эву минувших привлечь высоком Берет неслись трюфли Прелестным опасной мадригал приказала фальшивя согласно Онегину Дошло Крестьянин глубоко созрел тоске французский чужеземные верхом блаженство злых смелый приличию тревожить ствол пушистый Умел построже француз Qu'ecrirez-vous ободренный Невольно вежды почти 36 Устремлены обманы распущенные задремал волоса ведет инвалид родителей ввинченный игрушкой Взяв окном устарела спаситель заманчивой Надолго Смелей запущенный пойдет забав слеза
